In [4]:
import os
import glob
import json
import requests

import rasterio
from rasterio.errors import RasterioIOError
from rasterio.merge import merge
from shapely.geometry import box, mapping

# 0. SETTING PARAMETERS
lon_min, lon_max = -124.5, -114.0
lat_min, lat_max =   32.0,    42.0
geom = mapping(box(lon_min, lat_min, lon_max, lat_max))
date_range = "2020-08-15/2020-08-15"

out_root = r"D:\1Research\2025\NOAA_SatHack\data\sentinel_ca"
os.makedirs(out_root, exist_ok=True)

bands = ["B03", "B04", "B08", "B11"]
STAC_SEARCH = "https://earth-search.aws.element84.com/v0/search"

# 1. STAC SEARCH
payload = {
    "collections": ["sentinel-s2-l2a-cogs"],
    "intersects": geom,
    "datetime": date_range,
    "query": { "eo:cloud_cover": { "lt": 20 } },
    "limit": 100
}
resp = requests.post(STAC_SEARCH, json=payload)
resp.raise_for_status()
features = resp.json()["features"]
print(f"Discovered {len(features)} Sentinel-2 scenes")

# 2. DOWNLOAD LOOP WITH STREAM and FALLBACK
for feat in features:
    scene_id = feat["id"].replace(":", "_")
    scene_dir = os.path.join(out_root, scene_id)
    os.makedirs(scene_dir, exist_ok=True)

    for band in bands:
        href = feat["assets"][band]["href"]
        out_path = os.path.join(scene_dir, f"{scene_id}_{band}.tif")
        if os.path.exists(out_path):
            continue

        print(f"Retrieving {band} for {scene_id}")
        try:
            with rasterio.Env(CPL_VSIL_CURL_USE_HEAD="FALSE"):
                with rasterio.open(f"/vsicurl/{href}") as src:
                    arr = src.read(1)
                    profile = src.profile.copy()
            profile.update(driver="GTiff", compress="lzw")
            with rasterio.open(out_path, "w", **profile) as dst:
                dst.write(arr, 1)

        except (RasterioIOError, OSError) as e:
            print(f"  Stream error: {e}\n  Falling back to full download…")
            r = requests.get(href, stream=True)
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024*1024):
                    f.write(chunk)



Discovered 41 Sentinel-2 scenes
Retrieving B03 for S2A_10SCF_20200815_0_L2A
Retrieving B04 for S2A_10SCF_20200815_0_L2A
Retrieving B08 for S2A_10SCF_20200815_0_L2A
Retrieving B11 for S2A_10SCF_20200815_0_L2A
Retrieving B03 for S2A_10SDF_20200815_0_L2A
Retrieving B04 for S2A_10SDF_20200815_0_L2A
Retrieving B08 for S2A_10SDF_20200815_0_L2A
Retrieving B11 for S2A_10SDF_20200815_0_L2A
Retrieving B03 for S2A_10SEF_20200815_0_L2A
Retrieving B04 for S2A_10SEF_20200815_0_L2A
Retrieving B08 for S2A_10SEF_20200815_0_L2A
Retrieving B11 for S2A_10SEF_20200815_0_L2A
Retrieving B03 for S2A_10SCG_20200815_0_L2A
Retrieving B04 for S2A_10SCG_20200815_0_L2A
Retrieving B08 for S2A_10SCG_20200815_0_L2A
Retrieving B11 for S2A_10SCG_20200815_0_L2A
Retrieving B03 for S2A_10SDG_20200815_1_L2A
Retrieving B04 for S2A_10SDG_20200815_1_L2A
Retrieving B08 for S2A_10SDG_20200815_1_L2A
Retrieving B11 for S2A_10SDG_20200815_1_L2A
Retrieving B03 for S2A_10SEG_20200815_1_L2A
Retrieving B04 for S2A_10SEG_20200815_1_L2A


MemoryError: Unable to allocate 115. MiB for an array with shape (1, 10980, 10980) and data type uint8